In [11]:
import duckdb

con = duckdb.connect("../data/warehouse/olist.duckdb")


In [3]:
con.execute("SHOW TABLES").fetchall()


[('customers',),
 ('order_items',),
 ('orders',),
 ('payments',),
 ('products',),
 ('reviews',),
 ('sellers',)]

In [4]:
con.execute("DESCRIBE orders").fetchdf()


,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,customer_id,VARCHAR,YES,None,None,None
2,order_status,VARCHAR,YES,None,None,None
3,order_purchase_timestamp,VARCHAR,YES,None,None,None
4,order_approved_at,VARCHAR,YES,None,None,None
5,order_delivered_carrier_date,VARCHAR,YES,None,None,None
6,order_delivered_customer_date,VARCHAR,YES,None,None,None
7,order_estimated_delivery_date,VARCHAR,YES,None,None,None


In [13]:
con.execute("DESCRIBE order_items").fetchdf()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,order_item_id,BIGINT,YES,None,None,None
2,product_id,VARCHAR,YES,None,None,None
3,seller_id,VARCHAR,YES,None,None,None
4,shipping_limit_date,VARCHAR,YES,None,None,None
5,price,DOUBLE,YES,None,None,None
6,freight_value,DOUBLE,YES,None,None,None


In [14]:
con.execute("DESCRIBE payments").fetchdf()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,payment_sequential,BIGINT,YES,None,None,None
2,payment_type,VARCHAR,YES,None,None,None
3,payment_installments,BIGINT,YES,None,None,None
4,payment_value,DOUBLE,YES,None,None,None


In [9]:
con.execute("SELECT order_id, order_item_id, product_id FROM order_items LIMIT 5").fetchdf()

,order_id,order_item_id,product_id
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089


In [10]:
con.execute("SELECT order_id FROM orders LIMIT 5").fetchdf()

,order_id
0,e481f51cbdc54678b7cc49136f2d6af7
1,53cdb2fc8bc7dce0b6741e2150273451
2,47770eb9100c2d0c44946d9cf07ec65d
3,949d5b44dbf5de918fe9c16f97b45f8a
4,ad21c59c0840e6cb83a9ceb5573f8159


In [12]:
con.execute("""
SELECT order_id, COUNT(*) AS n_items
FROM order_items
GROUP BY order_id
HAVING COUNT(*) > 1
LIMIT 10
""").fetchdf()


,order_id,n_items
0,024bee763e8abbf0cd46fa7f440c41ea,2
1,032b0e60cda9e20b30fc4b3bf1079e50,3
2,03da04c1427d7b8bff38d76efca1029d,2
3,054df8d1a5ab3cf520be20c25d98f5f4,2
4,07d35f516238db3cf2e71da57744922d,2
5,08c30c203ecbdda69311c1dba85abfa6,3
6,0ad3fdda3076a1be91b13c89f2049266,2
7,0be2a2a52adfb0963b9bb86b3b668b90,3
8,0d3e9a1db3f3e32d92a74cf84605172d,2
9,0d931f4590e355bc7aedc79ca822cd82,4


In [15]:
con.execute("""SELECT
    order_id,
    COUNT(*) AS number_of_items,
    SUM(price) AS total_items_price,
    SUM(freight_value) AS total_freight
FROM order_items
GROUP BY order_id""").fetch_df()

,order_id,number_of_items,total_items_price,total_freight
0,000aed2e25dbad2f9ddb70584c5a2ded,1,144.00,8.77
1,0039500db36d50d7d0bfaf23723550f3,1,169.90,33.33
2,0071ee2429bc1efdc43aa3e073a5290e,1,179.98,12.46
3,00789ce015e7e5791c7914f32bb4fad4,1,154.00,14.83
4,007ec946d4f798caf2b43c9d44c56421,1,27.90,15.10
...,...,...,...,...
98661,ff1037c14208a18e7482c6c7a988799f,1,22.99,7.42
98662,ff27293d3d0221896d1bc2dd36b73ebf,1,58.23,11.73
98663,ffa2a7fdee9759a1c69f742f0a21d371,1,29.90,8.11
98664,ffdd767d77c3322d78b286b6e8ecaf15,1,49.90,13.01
